#**Машинное обучение ИБ-2024**

#**Домашнее задание 1.**
#Регрессия, KNN, LinearRegression.

В данной домашней работе мы будем строить модели для предсказания цены квартиры в России. Ниже приведено описание некоторых колонок набора данных.

date - дата публикации объявления

price - цена в рублях

level- этаж, на котором находится квартира

levels - количество этажей в квартире

rooms - количество комнат в квартире. Если значение -1, то квартира считается апартаментами.

area - площадь квартиры.

kitchen_area - площадь кухни.

geo_lat - Latitude

geo_lon - Longitude

building_type - материал застройки. 0 - Don't know. 1 - Other. 2 - Panel. 3 - Monolithic. 4 - Brick. 5 - Blocky. 6 - Wooden

#Часть 0. Начало работы

Для начала работы с данными импортируем библиотеки, которые понадобятся в данном задании.

In [84]:
import math
import pandas as pd
import numpy as np
import matplotlib as plt
import sklearn
import seaborn as sns

Загрузим библиотеку folium для отображения данных на карте по координатам.

In [85]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


Распакуем наши данные из архива.

In [86]:
!unzip ...

unzip:  cannot find or open ..., ....zip or ....ZIP.


Загрузим данные из csv файла в датафрейм.

In [87]:
df = pd.read_csv('input_data.csv', sep=';', header=0, na_values=['NaN', ''])
df = df.iloc[::10, :]
df.head()

,date,price,level,levels,rooms,area,kitchen_area,geo_lat,geo_lon,building_type,object_type,postal_code,street_id,id_region,house_id
0,2021-01-01,2451300,15,31,1,30.3,0.0,56.780112,60.699355,0,2,620000.0,NaN,66,1632918.0
10,2021-01-01,3350000,1,19,-1,10.1,2.0,55.778026,37.540147,3,0,123007.0,580339.0,77,2069062.0
20,2021-01-01,2435000,5,5,1,49.7,0.0,54.603336,36.267340,4,0,248031.0,564060.0,40,2657892.0
30,2021-01-01,900000,3,4,3,54.0,-100.0,51.911466,102.435598,4,0,671023.0,153245.0,3,632956.0
40,2021-01-01,2370000,3,4,2,54.0,0.0,43.047672,44.671295,0,0,362021.0,194558.0,15,2275604.0


Отобразим на карте координаты наших построек.

In [88]:
import folium
from IPython.display import display

map_df = df.iloc[:1000]

m = folium.Map(location=[55.751244, 37.618423], zoom_start=10)

lats = map_df['geo_lat'].loc[:1000]
longs = map_df['geo_lon'].loc[:1000]
for point in zip(lats, longs):
    folium.Marker(
        location=[point[0], point[1]]
    ).add_to(m)

display(m)

# Часть 1. Подготовим данные для обработки моделями машинного обучения.

**0.5 Балл**. География наших наблюдений в наборе данных крайне большая. Однако мы знаем, что стоимость квартир в Москве и Санкт-Петербурге намного выше, чем в среднем по России. Давайте сделаем признаки, который показывают, находится ли квартира в 20 килиметрах от центра Москвы или находится ли квартира в 20 килиметрах от центра Санкт-Петербурга.

Создайте два признака is_Moscow и is_Saint_Peterburg. Для нахождения расстояния по координатам используйте функцию haversine_distance.

In [89]:
import numpy as np

def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371.0
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi / 2) ** 2 + \
        np.cos(phi1) * np.cos(phi2) * \
        np.sin(delta_lambda / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    distance = R * c
    return distance


**0.5 Балла**. В нашем наборе данных есть признаки, которые мы теоретически можем использовать, например postal_code, но мы это будем делать в рамках домашней работы очень-очень долго. Поэтому предлагается удалить ненужные признаки из датафрейма.

Удалим geo_lat,	geo_lon,	object_type,	postal_code,	street_id,	id_region,	house_id.

In [90]:
df = df.drop(['geo_lat', 'geo_lon', 'object_type', 'postal_code', 'street_id', 'id_region', 'house_id'], axis=1)


**0.5 Балл**. Для начала Вам предлагается проанализировать Ваши оставшиеся признаки (колонки) в наборе данных. Какие колонки категориальные? Какие числовые?

Категориальные: (Ваш ответ)

Числовые: (Ваш ответ)

Давайте закодируем категориальные признаки с помощью OneHot-Encoding. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [91]:
from sklearn.preprocessing import OneHotEncoder

categorical_features = ['building_type', 'rooms']

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

encoded_cats = encoder.fit_transform(df[categorical_features])

encoded_df = pd.DataFrame(encoded_cats, columns=encoder.get_feature_names_out(categorical_features))

df = df.join(encoded_df).drop(categorical_features, axis=1)


**0.5 Балл**. Поработаем с числовыми признаками:


1.   Добавьте в ваш датасет два признака: количество дней со дня первого наблюдения (разница между датами объявлений). Возможно, для предсказания цены не так важен этаж, как важно отношение этажа квартиры на количество этажей в доме, добавьте этот признак. После добавления нового признака колонку date можно удалить.
2.   Числовые признаки могут иметь разные порядки. Давайте отнормируем числовые признаки с помощью StandartScaller https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html.



In [92]:

df['date'] = pd.to_datetime(df['date'])

df['days_since_first'] = (df['date'] - df['date'].min()).dt.days

df = df.drop('date', axis=1)


In [93]:
df['floor_ratio'] = df['level'] / df['levels']

In [94]:
from sklearn.preprocessing import StandardScaler


numerical_features = ['price', 'level', 'levels', 'area', 'kitchen_area', 'days_since_first', 'floor_ratio']
df[numerical_features] = df[numerical_features].replace([np.inf, -np.inf], np.nan)
scaler = StandardScaler()
df[numerical_features] = scaler.fit_transform(df[numerical_features])

**2 Балла**. Реализуйте класс KNNRegressor, который должен делать регрессию методом k ближайших соседей.

In [95]:
import numpy as np

class KNNRegressor:
    def __init__(self, n_neighbors=5, metric='euclidean'):
        self.n_neighbors = n_neighbors
        self.metric = metric

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def _compute_distances(self, x):
        if self.metric == 'euclidean':
            distances = np.sqrt(np.sum((self.X_train - x) ** 2, axis=1))
        else:
            raise ValueError(f"Unsupported metric: {self.metric}")
        return distances

    def predict(self, X):
        y_pred = []
        for x in X:
            distances = self._compute_distances(x)
            neighbors_idx = np.argsort(distances)[:self.n_neighbors]
            y_pred.append(np.mean(self.y_train[neighbors_idx]))
        return np.array(y_pred)


**3 Балла**. Реализуйте класс LinearRegression, поддерживающий обучение градиентными спусками SGD, Momentum, AdaGrad. Используйте градиент для оптимизации функции потерь MSE.

In [96]:
import numpy as np

class LinearRegression:
    def __init__(self, learning_rate=0.01, optimization='SGD', epsilon=1e-8, decay_rate=0.9, max_iter=1000):
        self.learning_rate = learning_rate
        self.optimization = optimization
        self.epsilon = epsilon
        self.decay_rate = decay_rate
        self.max_iter = max_iter
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        if self.optimization == 'Momentum':
            v_w = np.zeros(n_features)
            v_b = 0
        elif self.optimization == 'AdaGrad':
            G_w = np.zeros(n_features)
            G_b = 0

        for _ in range(self.max_iter):
            y_pred = np.dot(X, self.weights) + self.bias
            errors = y_pred - y
            dw = (2 / n_samples) * np.dot(X.T, errors)
            db = (2 / n_samples) * np.sum(errors)

            if self.optimization == 'SGD':
                self.weights -= self.learning_rate * dw
                self.bias -= self.learning_rate * db
            elif self.optimization == 'Momentum':
                v_w = self.decay_rate * v_w + self.learning_rate * dw
                v_b = self.decay_rate * v_b + self.learning_rate * db
                self.weights -= v_w
                self.bias -= v_b
            elif self.optimization == 'AdaGrad':
                G_w += dw ** 2
                G_b += db ** 2
                adjusted_lr_w = self.learning_rate / (np.sqrt(G_w) + self.epsilon)
                adjusted_lr_b = self.learning_rate / (np.sqrt(G_b) + self.epsilon)
                self.weights -= adjusted_lr_w * dw
                self.bias -= adjusted_lr_b * db
            else:
                raise ValueError(f"Unsupported optimization method: {self.optimization}")

    def predict(self, X):
        return np.dot(X, self.weights) + self.bias


# Часть 2. Эксперименты с моделями машинного обучения.

**3 Балла**. Проведите эксперименты с написанными Вами методами машинного обучения. Выделите обучающую и тестовую выборки в отношении 0,8 и 0,2 соответственно. Измерьте ошибку MSE, MAE, RMSE. Заиспользуйте методы KNNRegressor и LinearRegression из библиотеки sklearn, сравните качество Ваших решений и библиотечных.

In [97]:
from sklearn.model_selection import train_test_split

X = df.drop('price', axis=1).values
y = df['price'].values


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

nan_mask_train = ~np.isnan(X_train).any(axis=1)  
X_train = X_train[nan_mask_train]  
y_train = y_train[nan_mask_train]  

nan_mask_test = ~np.isnan(X_test).any(axis=1)
X_test = X_test[nan_mask_test]
y_test = y_test[nan_mask_test]



In [98]:
knn_regressor = KNNRegressor(n_neighbors=5)
knn_regressor.fit(X_train, y_train)
y_pred_knn = knn_regressor.predict(X_test)
print("knn_regressor")

lin_reg_sgd = LinearRegression(learning_rate=0.01, optimization='SGD', max_iter=1000)
lin_reg_sgd.fit(X_train, y_train)
y_pred_lin_sgd = lin_reg_sgd.predict(X_test)
print("lin_reg_sgd")

lin_reg_momentum = LinearRegression(learning_rate=0.01, optimization='Momentum', decay_rate=0.9, max_iter=1000)
lin_reg_momentum.fit(X_train, y_train)
y_pred_lin_momentum = lin_reg_momentum.predict(X_test)
print("lin_reg_momentum")

lin_reg_adagrad = LinearRegression(learning_rate=0.01, optimization='AdaGrad', epsilon=1e-8, max_iter=1000)
lin_reg_adagrad.fit(X_train, y_train)
y_pred_lin_adagrad = lin_reg_adagrad.predict(X_test)
print("lin_reg_adagrad")

knn_regressor
lin_reg_sgd
lin_reg_momentum
lin_reg_adagrad


In [99]:
from sklearn.neighbors import KNeighborsRegressor

knn_sklearn = KNeighborsRegressor(n_neighbors=5)
knn_sklearn.fit(X_train, y_train)
y_pred_knn_sklearn = knn_sklearn.predict(X_test)


In [100]:
from sklearn.linear_model import LinearRegression as SklearnLinearRegression

lin_reg_sklearn = SklearnLinearRegression()
lin_reg_sklearn.fit(X_train, y_train)
y_pred_lin_sklearn = lin_reg_sklearn.predict(X_test)


In [101]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

def evaluate_model(y_true, y_pred, model_name):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    print(f"{model_name}: MSE={mse:.4f}, MAE={mae:.4f}, RMSE={rmse:.4f}")


In [102]:

evaluate_model(y_test, y_pred_knn, 'KNNRegressor')

evaluate_model(y_test, y_pred_lin_sgd, 'LinearRegression SGD')

evaluate_model(y_test, y_pred_lin_momentum, 'LinearRegression Momentum')

evaluate_model(y_test, y_pred_lin_adagrad, 'LinearRegression AdaGrad')


KNNRegressor: MSE=0.6322, MAE=0.0764, RMSE=0.7951
LinearRegression SGD: MSE=0.5532, MAE=0.0962, RMSE=0.7438
LinearRegression Momentum: MSE=0.5532, MAE=0.0963, RMSE=0.7438
LinearRegression AdaGrad: MSE=0.5532, MAE=0.0963, RMSE=0.7438


In [103]:
evaluate_model(y_test, y_pred_knn_sklearn, 'KNeighborsRegressor')

evaluate_model(y_test, y_pred_lin_sklearn, 'LinearRegression')


KNeighborsRegressor: MSE=0.6322, MAE=0.0764, RMSE=0.7951
LinearRegression: MSE=0.5532, MAE=0.0963, RMSE=0.7438
